### Initialise

In [2]:
## Import necessary packages
import numpy as np; 
import matplotlib.pyplot as plt; 
import pandas as pd; 
from sklearn.preprocessing import MinMaxScaler; 
from sklearn.model_selection import train_test_split

### Preprocess Data

In [5]:
data = pd.read_csv('data/clean_data.csv'); 
print(data.columns, "\n\n" ,data.shape); 
data.head()

Index(['Time (h)', 'Aeration rate(Fg:L/h)', 'Sugar feed rate(Fs:L/h)',
       'Acid flow rate(Fa:L/h)', 'Base flow rate(Fb:L/h)',
       'Heating/cooling water flow rate(Fc:L/h)',
       'Heating water flow rate(Fh:L/h)',
       'Water for injection/dilution(Fw:L/h)',
       'Air head pressure(pressure:bar)', 'Dumped broth flow(Fremoved:L/h)',
       'Substrate concentration(S:g/L)',
       'Dissolved oxygen concentration(DO2:mg/L)',
       'Penicillin concentration(P:g/L)', 'Vessel Volume(V:L)', 'pH(pH:pH)',
       'Temperature(T:K)', 'Generated heat(Q:kJ)',
       'carbon dioxide percent in off-gas(CO2outgas:%)',
       'PAA flow(Fpaa:PAA flow (L/h))', 'Oil flow(Foil:L/hr)',
       'Oxygen Uptake Rate(OUR:(g min^{-1}))',
       'Oxygen in percent in off-gas(O2:O2  (%))',
       'Fault reference(Fault_ref:Fault ref)',
       '0 - Recipe driven 1 - Operator controlled(Control_ref:Control ref)'],
      dtype='object') 

 (113935, 24)


,Time (h),Aeration rate(Fg:L/h),Sugar feed rate(Fs:L/h),Acid flow rate(Fa:L/h),Base flow rate(Fb:L/h),Heating/cooling water flow rate(Fc:L/h),Heating water flow rate(Fh:L/h),Water for injection/dilution(Fw:L/h),Air head pressure(pressure:bar),Dumped broth flow(Fremoved:L/h),...,pH(pH:pH),Temperature(T:K),Generated heat(Q:kJ),carbon dioxide percent in off-gas(CO2outgas:%),PAA flow(Fpaa:PAA flow (L/h)),Oil flow(Foil:L/hr),Oxygen Uptake Rate(OUR:(g min^{-1})),Oxygen in percent in off-gas(O2:O2 (%)),Fault reference(Fault_ref:Fault ref),0 - Recipe driven 1 - Operator controlled(Control_ref:Control ref)
0,0.2,30.0,8.0,0.0000,30.118,9.8335,0.0001,0.0,0.6,0.0,...,6.4472,298.22,0.000001,0.089514,5.0,22.0,0.480510,0.19595,0.0,0.0
1,0.4,30.0,8.0,0.0000,51.221,18.1550,0.0001,0.0,0.6,0.0,...,6.4932,298.17,0.000001,0.101760,5.0,22.0,0.058147,0.20390,0.0,0.0
2,0.6,30.0,8.0,0.0000,54.302,9.5982,0.0001,0.0,0.6,0.0,...,6.5425,298.14,0.000001,0.105800,5.0,22.0,-0.041505,0.20575,0.0,0.0
3,0.8,30.0,8.0,0.0000,37.816,4.3395,0.0001,0.0,0.6,0.0,...,6.5753,298.11,0.000001,0.108190,5.0,22.0,-0.056737,0.20602,0.0,0.0
4,1.0,30.0,8.0,0.5181,18.908,1.1045,0.0001,0.0,0.6,0.0,...,6.5825,298.09,0.000001,0.110300,5.0,22.0,-0.049975,0.20589,0.0,0.0


In [6]:
# Scaling from 0-1
scaler = MinMaxScaler(); 

d_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns); 
d_scaled.head()

,Time (h),Aeration rate(Fg:L/h),Sugar feed rate(Fs:L/h),Acid flow rate(Fa:L/h),Base flow rate(Fb:L/h),Heating/cooling water flow rate(Fc:L/h),Heating water flow rate(Fh:L/h),Water for injection/dilution(Fw:L/h),Air head pressure(pressure:bar),Dumped broth flow(Fremoved:L/h),...,pH(pH:pH),Temperature(T:K),Generated heat(Q:kJ),carbon dioxide percent in off-gas(CO2outgas:%),PAA flow(Fpaa:PAA flow (L/h)),Oil flow(Foil:L/hr),Oxygen Uptake Rate(OUR:(g min^{-1})),Oxygen in percent in off-gas(O2:O2 (%)),Fault reference(Fault_ref:Fault ref),0 - Recipe driven 1 - Operator controlled(Control_ref:Control ref)
0,0.000000,0.181818,0.040541,0.000000,0.133858,0.006556,0.0,0.0,0.0,1.0,...,0.767126,0.258427,0.0,0.002000,0.333333,0.0,0.215230,0.677992,0.0,0.0
1,0.000690,0.181818,0.040541,0.000000,0.227649,0.012103,0.0,0.0,0.0,1.0,...,0.800686,0.249064,0.0,0.003738,0.333333,0.0,0.162455,0.759106,0.0,0.0
2,0.001380,0.181818,0.040541,0.000000,0.241342,0.006399,0.0,0.0,0.0,1.0,...,0.836653,0.243446,0.0,0.004311,0.333333,0.0,0.150004,0.777982,0.0,0.0
3,0.002070,0.181818,0.040541,0.000000,0.168071,0.002893,0.0,0.0,0.0,1.0,...,0.860582,0.237828,0.0,0.004650,0.333333,0.0,0.148100,0.780737,0.0,0.0
4,0.002761,0.181818,0.040541,0.039866,0.084036,0.000736,0.0,0.0,0.0,1.0,...,0.865835,0.234082,0.0,0.004950,0.333333,0.0,0.148945,0.779410,0.0,0.0


In [15]:
# Split into inputs and outputs
inputs_keys = [
    "Time (h)", "Aeration rate(Fg:L/h)", "Sugar feed rate(Fs:L/h)","Acid flow rate(Fa:L/h)",
    "Base flow rate(Fb:L/h)","Heating/cooling water flow rate(Fc:L/h)","Heating water flow rate(Fh:L/h)",
    "Water for injection/dilution(Fw:L/h)","Substrate concentration(S:g/L)","PAA flow(Fpaa:PAA flow (L/h))",
    "Oil flow(Foil:L/hr)", "0 - Recipe driven 1 - Operator controlled(Control_ref:Control ref)",
    "Oxygen Uptake Rate(OUR:(g min^{-1}))"
]
inputs = d_scaled[inputs_keys]
print(len(inputs_keys)," keys in ", len(inputs.columns), "cols")
inputs.head()

13  keys in  13 cols


,Time (h),Aeration rate(Fg:L/h),Sugar feed rate(Fs:L/h),Acid flow rate(Fa:L/h),Base flow rate(Fb:L/h),Heating/cooling water flow rate(Fc:L/h),Heating water flow rate(Fh:L/h),Water for injection/dilution(Fw:L/h),Substrate concentration(S:g/L),PAA flow(Fpaa:PAA flow (L/h)),Oil flow(Foil:L/hr),0 - Recipe driven 1 - Operator controlled(Control_ref:Control ref),Oxygen Uptake Rate(OUR:(g min^{-1}))
0,0.000000,0.181818,0.040541,0.000000,0.133858,0.006556,0.0,0.0,0.008306,0.333333,0.0,0.0,0.215230
1,0.000690,0.181818,0.040541,0.000000,0.227649,0.012103,0.0,0.0,0.008715,0.333333,0.0,0.0,0.162455
2,0.001380,0.181818,0.040541,0.000000,0.241342,0.006399,0.0,0.0,0.009107,0.333333,0.0,0.0,0.150004
3,0.002070,0.181818,0.040541,0.000000,0.168071,0.002893,0.0,0.0,0.009492,0.333333,0.0,0.0,0.148100
4,0.002761,0.181818,0.040541,0.039866,0.084036,0.000736,0.0,0.0,0.009864,0.333333,0.0,0.0,0.148945


In [16]:
outputs_keys = list(set(d_scaled.columns) - set(inputs_keys))
outputs = d_scaled[outputs_keys]
print(len(outputs_keys)," keys in ", len(outputs.columns), "cols")
outputs.head()

11  keys in  11 cols


,Oxygen in percent in off-gas(O2:O2 (%)),Fault reference(Fault_ref:Fault ref),Generated heat(Q:kJ),Penicillin concentration(P:g/L),Vessel Volume(V:L),Dissolved oxygen concentration(DO2:mg/L),Temperature(T:K),Air head pressure(pressure:bar),Dumped broth flow(Fremoved:L/h),pH(pH:pH),carbon dioxide percent in off-gas(CO2outgas:%)
0,0.677992,0.0,0.0,1.791808e-27,0.049276,0.884124,0.258427,0.0,1.0,0.767126,0.002000
1,0.759106,0.0,0.0,2.763729e-05,0.049480,0.883351,0.249064,0.0,1.0,0.800686,0.003738
2,0.777982,0.0,0.0,2.761905e-05,0.049685,0.882512,0.243446,0.0,1.0,0.836653,0.004311
3,0.780737,0.0,0.0,2.760247e-05,0.049787,0.880900,0.237828,0.0,1.0,0.860582,0.004650
4,0.779410,0.0,0.0,2.758782e-05,0.049838,0.879095,0.234082,0.0,1.0,0.865835,0.004950
